In [2]:
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import jieba as jb
#from sklearn.externals import joblib
from sklearn.svm import SVC
from gensim.models.word2vec import Word2Vec


In [3]:
# 数据读取evaluation列
X = pd.read_csv('data.csv', encoding='utf-8', usecols=['evaluation'])

print(X.head(5))

# 数据读取label列
y = pd.read_csv('data.csv', encoding='utf-8', usecols=['label'])

print(y.head(5))
# 数据分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

                                          evaluation
0                                    用了一段时间，感觉还不错，可以
1  电视非常好，已经是家里的第二台了。第一天下单，第二天就到本地了，可是物流的人说车坏了，一直催...
2                  电视比想象中的大好多，画面也很清晰，系统很智能，更多功能还在摸索中
3                                                 不错
4     用了这么多天了，感觉还不错。夏普的牌子还是比较可靠。希望以后比较耐用，现在是考量质量的时候。
   label
0      1
1      1
2      1
3      1
4      1


In [4]:
#对句子中的所有词向量取均值，来生成一个句子的vec
def build_vector(text,size,wv):
    #创建一个指定大小的数据空间
    vec = np.zeros(size).reshape((1,size))
    #count是统计有多少词向量
    count = 0
    #循环所有的词向量进行求和
    for w in text:
        try:
            vec +=  wv[w].reshape((1,size))
            count +=1
        except:
            continue
        
    #循环完成后求均值
    if count!=0:
        vec/=count
    return vec


#分词
X_train1 = [jb.lcut(x) for x in X_train.evaluation]
X_test1 = [jb.lcut(x) for x in X_test.evaluation]


#初始化模型和词表
wv = Word2Vec(vector_size=300,min_count=10)
wv.build_vocab(X_train1)

# 训练并建模
wv.train(X_train1,epochs=1,total_examples=1)
#获取train_vecs
train_vecs = np.concatenate([build_vector(z,300,wv) for z in X_train1])
#保存处理后的词向量
np.save('data/train_vecs.npy',train_vecs)
#保存模型
wv.save("data/model3.model")

wv.train(X_test1,total_examples=1, epochs=1)
test_vecs = np.concatenate([build_vector(z,300,wv) for z in X_test1])
np.save('data/test_vecs.npy',test_vecs)


Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\wangzihan\AppData\Local\Temp\jieba.cache
Loading model cost 0.616 seconds.
Prefix dict has been built successfully.


In [5]:
# 训练SVM模型

import joblib
from sklearn.metrics import classification_report


clf = SVC(kernel='rbf', verbose=True)

clf.fit(train_vecs, y_train)

# 保存模型

joblib.dump(clf, 'data/svm_model.model')

# 评估模型


print(classification_report(y_test, clf.predict(test_vecs)))

c:\Users\wangzihan\AppData\Local\Programs\Python\Python36\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


[LibSVM]              precision    recall  f1-score   support

           0       0.54      1.00      0.70       460
           1       0.00      0.00      0.00       397

    accuracy                           0.54       857
   macro avg       0.27      0.50      0.35       857
weighted avg       0.29      0.54      0.37       857



c:\Users\wangzihan\AppData\Local\Programs\Python\Python36\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\wangzihan\AppData\Local\Programs\Python\Python36\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\wangzihan\AppData\Local\Programs\Python\Python36\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf